# Motivation

Before walking you through the project, let me tell you a little bit about the motivation behind it. Aside from Data Science and Machine Learning, my other passion is spending time in the mountains. Planning a trip to any mountain requires lots of careful planning in order to minimize the risks. That means paying close attention to the weather conditions as the summit day approaches. My absolutely favorite website for this is Mountain-Forecast.com which gives you the weather forecasts for almost any mountain in the world at different elevations. The only problem is that it doesn’t offer any historical data (as far as I can tell), which can sometimes be useful when determining if it is a good idea to make the trip or wait for better conditions.
This problem has been in the back of my mind for a while and I finally decided to do something about it. Below, I’ll describe how I wrote a web scraper for Mountain-Forecast.com using Python and Beautiful Soup, and put it into a Raspberry Pi to collect the data on a daily basis.

# I. Scraping with Beautiful Soup and Selenium

In [17]:
import os
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import numpy as np
import itertools as it
import random
import time
from fake_useragent import UserAgent
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import sys
import regex as re

In [19]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
chromedriver = "C:/Users/jerem/Anaconda3/envs/environment-deep-learning-cookbook/Lib/site-packages/notebook/tests/selenium/chromedriver_win32/chromedriver" # path to the chromedriver executable
chromedriver = os.path.expanduser(chromedriver)
print('chromedriver path: {}'.format(chromedriver))
sys.path.append(chromedriver)

driver = webdriver.Chrome(chromedriver)

chromedriver path: C:/Users/jerem/Anaconda3/envs/environment-deep-learning-cookbook/Lib/site-packages/notebook/tests/selenium/chromedriver_win32/chromedriver


In [3]:
seloger_toulouse_url = 'https://www.seloger.com/immobilier/locations/immo-toulouse-31/bien-appartement/?LISTING-LISTpg='

def get_page_links(url, number_of_pages):
    page_links=[] # Create a list of pages links
    for i in range(1,number_of_pages+1):
        j = url + str(i)
        page_links.append(j)
    return page_links

#page_links = get_page_links(seloger_toulouse_url,3) 

In [4]:
def get_appartment_links(pages, driver):
    
    # Setting a list of listings links
    appartment_links=[] 
    
    # Getting length of list 
    length = len(pages) 
    
    while len(pages) > 0: 
        for i in pages:
            
            #print('Extracting links from page',pages.index(i)+1,'out of',len(pages),'left')
            # we try to access a page with the new proxy
            try:
                driver.get(i)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Extract links information via the find_all function of the soup object 
                listings = soup.find_all("a", attrs={"name": "classified-link"})
                page_data = [row['href'] for row in listings]
                appartment_links.append(page_data) # list of listings links
                
                pages.remove(i)

                print('There are',len(pages),'pages left to examine')
                time.sleep(random.randrange(11,21))
    
            except:
                print("Skipping. Connnection error")
                time.sleep(random.randrange(300,600))
                
    return appartment_links

In [5]:
# Create a flatten function:
def flatten_list(appartment_links):
    appartment_links_flat=[]
    for sublist in appartment_links:
        for item in sublist:
            appartment_links_flat.append(item)
    return appartment_links_flat
        
#or appartment_links_flat = list(it.chain.from_iterable(appartment_links))

#appartment_links_flat = flatten_list(appartment_links)
# Check the number of links
#len(appartment_links_flat)

In [6]:
def get_title(soup):
    try:
        title = soup.title.text
        return title
    except:
        return np.nan

    
def get_agency(soup):
    try:
        agency = soup.find_all("a", class_="agence-link")
        agency2 = agency[0].text
        agency3 = agency2.replace('\n', '').lower()
        return agency3
    except:
        return np.nan
    
    
def get_housing_type(soup):
    try:
        ht = soup.find_all("h2", class_="c-h2")
        ht2 = ht[0].text
        return ht2
    except:
        return np.nan
    
    
def get_city(soup):
    try:
        city = soup.find_all("p", class_="localite")
        city2 = city[0].text
        return city2
    except:
        return np.nan
    
    
def get_details(soup):
    try:
        details = soup.find_all("h1", class_="detail-title title1")
        details2 = details[0].text
        details3 = details2.replace('\n', '').lower()
        return details3
    except:
        return np.nan
    
    
def get_rent(soup):
    try:
        rent = soup.find_all("a", class_="js-smooth-scroll-link price")
        rent2 = rent[0].text
        rent3 = int(''.join(filter(str.isdigit, rent2)))
        return rent3
    except:
        return np.nan
    
def get_charges(soup):
    try:
        cha = soup.find_all("sup", class_="u-thin u-300 u-black-snow")
        cha2 = cha[0].text
        return cha2
    except:
        return np.nan
    
    
def get_rent_info(soup):
    try:
        rent_info = soup.find_all("section", class_="categorie with-padding-bottom")
        rent_info2 = rent_info[0].find_all("p", class_="sh-text-light")
        rent_info3 = rent_info2[0].text
        return rent_info3
    except:
        return 'None'
    
    
def get_criteria(soup):
    try:
        crit = soup.find_all("section", class_="categorie")
        crit2 = [div.text for ul in crit for div in ul.find_all("div", class_="u-left")]
        crit3=(" ; ".join(crit2)) # concatenate string items in a list into a single string
        return crit3
    except:
        return 'None'
    
    
def get_energy_rating(soup):
    try:
        ener = soup.find_all("div", class_="info-detail")
        ener2 = ener[0].text
        ener3 = int(''.join(filter(str.isdigit, ener2)))
        return ener3
    except:
        return np.nan
    
    
def get_gas_rating(soup):
    try:
        gas = soup.find_all("div", class_="info-detail")
        gas2 = gas[1].text
        gas3 = int(''.join(filter(str.isdigit, gas2)))
        return gas3
    except:
        return np.nan
    
    
def get_description(soup):
    try:
        descr = soup.find_all(class_="sh-text-light")
        descr2 = descr[0].text
        return descr2
    except:
        return 'None'

In [7]:
def get_html_data(url, driver):
    driver.get(url)
    #time.sleep(random.lognormal(0,1))
    time.sleep(random.randrange(5,15))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

In [8]:
def get_appartment_data(driver,links):
    appartment_data = []

    while len(links) > 0: 
        for i in links:

            print('Now extracting data from listing {} out of {}.'.format(links.index(i)+1,len(links)))

            # we try to access a page with the new proxy
            try:
                soup = get_html_data(i,driver)
                title = get_title(soup)
                agency = get_agency(soup)
                housing_type = get_housing_type(soup)
                city = get_city(soup)
                details = get_details(soup)
                rent = get_rent(soup)
                charges = get_charges(soup)
                rent_info = get_rent_info(soup)
                criteria = get_criteria(soup)
                energy_rating = get_energy_rating(soup)
                gas_rating = get_gas_rating(soup)
                description = get_description(soup)

                # if listings is not available anymore then remove the listing from the list
                if title == 'Location appartements Toulouse (31) | Louer appartements à Toulouse 31000': 
                    print('This appartment is no longer available.')   
                    links.remove(i)

                # if listing not accessible (robot) then go to the next one and try again later
                elif pd.isna(housing_type) == True and pd.isna(city) == True and pd.isna(rent) == True: 
                    print('You Shall Not Pass!')                    
                    time.sleep(random.randrange(300,600))

                # if access to the listing granted then extract data and remove the listing from the list
                else:
                    appartment_data.append([i,title,agency,housing_type,city,details,rent,charges,rent_info,
                                            criteria,energy_rating,gas_rating,description]) 
                    links.remove(i)
                    print('Good! There are {} listings left to examine.'.format(len(links)))

            except:
                print("Skipping. Connnection error")
                time.sleep(random.randrange(60,120))
     
    
    df = pd.DataFrame(appartment_data,columns = ['link','title','agency','housing_type','city','details','rent','charges','rent_info',
                                            'criteria','energy_rating','gas_rating','description'])
    
    return df

In [9]:
page_links = get_page_links(seloger_toulouse_url,96) 
appartment_links = get_appartment_links(page_links,driver)
appartment_links_flat = flatten_list(appartment_links)
df_appartment = get_appartment_data(driver,appartment_links_flat)

There are 95 pages left to examine
There are 94 pages left to examine
There are 93 pages left to examine
There are 92 pages left to examine
There are 91 pages left to examine
There are 90 pages left to examine
There are 89 pages left to examine
There are 88 pages left to examine
There are 87 pages left to examine
There are 86 pages left to examine
There are 85 pages left to examine
There are 84 pages left to examine
There are 83 pages left to examine
There are 82 pages left to examine
There are 81 pages left to examine
There are 80 pages left to examine
There are 79 pages left to examine
There are 78 pages left to examine
There are 77 pages left to examine
There are 76 pages left to examine
There are 75 pages left to examine
There are 74 pages left to examine
There are 73 pages left to examine
There are 72 pages left to examine
There are 71 pages left to examine
There are 70 pages left to examine
There are 69 pages left to examine
There are 68 pages left to examine
There are 67 pages l

Good! There are 2323 listings left to examine.
Now extracting data from listing 53 out of 2323.
Good! There are 2322 listings left to examine.
Now extracting data from listing 54 out of 2322.
Good! There are 2321 listings left to examine.
Now extracting data from listing 55 out of 2321.
Good! There are 2320 listings left to examine.
Now extracting data from listing 56 out of 2320.
Good! There are 2319 listings left to examine.
Now extracting data from listing 57 out of 2319.
Good! There are 2318 listings left to examine.
Now extracting data from listing 58 out of 2318.
Good! There are 2317 listings left to examine.
Now extracting data from listing 59 out of 2317.
Good! There are 2316 listings left to examine.
Now extracting data from listing 60 out of 2316.
Good! There are 2315 listings left to examine.
Now extracting data from listing 61 out of 2315.
Good! There are 2314 listings left to examine.
Now extracting data from listing 62 out of 2314.
Good! There are 2313 listings left to ex

Now extracting data from listing 140 out of 2245.
You Shall Not Pass!
Now extracting data from listing 141 out of 2245.
You Shall Not Pass!
Now extracting data from listing 142 out of 2245.
You Shall Not Pass!
Now extracting data from listing 143 out of 2245.
You Shall Not Pass!
Now extracting data from listing 144 out of 2245.
You Shall Not Pass!
Now extracting data from listing 145 out of 2245.
You Shall Not Pass!
Now extracting data from listing 146 out of 2245.
You Shall Not Pass!
Now extracting data from listing 147 out of 2245.
Good! There are 2244 listings left to examine.
Now extracting data from listing 148 out of 2244.
Good! There are 2243 listings left to examine.
Now extracting data from listing 149 out of 2243.
Good! There are 2242 listings left to examine.
Now extracting data from listing 150 out of 2242.
Good! There are 2241 listings left to examine.
Now extracting data from listing 151 out of 2241.
Good! There are 2240 listings left to examine.
Now extracting data from 

Good! There are 2165 listings left to examine.
Now extracting data from listing 227 out of 2165.
Good! There are 2164 listings left to examine.
Now extracting data from listing 228 out of 2164.
Good! There are 2163 listings left to examine.
Now extracting data from listing 229 out of 2163.
Good! There are 2162 listings left to examine.
Now extracting data from listing 230 out of 2162.
Good! There are 2161 listings left to examine.
Now extracting data from listing 231 out of 2161.
Good! There are 2160 listings left to examine.
Now extracting data from listing 232 out of 2160.
Good! There are 2159 listings left to examine.
Now extracting data from listing 233 out of 2159.
Good! There are 2158 listings left to examine.
Now extracting data from listing 234 out of 2158.
Good! There are 2157 listings left to examine.
Now extracting data from listing 235 out of 2157.
Good! There are 2156 listings left to examine.
Now extracting data from listing 236 out of 2156.
Good! There are 2155 listings 

Good! There are 2082 listings left to examine.
Now extracting data from listing 313 out of 2082.
Good! There are 2081 listings left to examine.
Now extracting data from listing 314 out of 2081.
Good! There are 2080 listings left to examine.
Now extracting data from listing 315 out of 2080.
Good! There are 2079 listings left to examine.
Now extracting data from listing 316 out of 2079.
Good! There are 2078 listings left to examine.
Now extracting data from listing 317 out of 2078.
Good! There are 2077 listings left to examine.
Now extracting data from listing 318 out of 2077.
Good! There are 2076 listings left to examine.
Now extracting data from listing 319 out of 2076.
Good! There are 2075 listings left to examine.
Now extracting data from listing 320 out of 2075.
Good! There are 2074 listings left to examine.
Now extracting data from listing 321 out of 2074.
Good! There are 2073 listings left to examine.
Now extracting data from listing 322 out of 2073.
Good! There are 2072 listings 

...

Good! There are 422 listings left to examine.
Now extracting data from listing 344 out of 422.
Good! There are 421 listings left to examine.
Now extracting data from listing 345 out of 421.
Good! There are 420 listings left to examine.
Now extracting data from listing 346 out of 420.
Good! There are 419 listings left to examine.
Now extracting data from listing 347 out of 419.
Good! There are 418 listings left to examine.
Now extracting data from listing 348 out of 418.
Good! There are 417 listings left to examine.
Now extracting data from listing 349 out of 417.
Good! There are 416 listings left to examine.
Now extracting data from listing 350 out of 416.
Good! There are 415 listings left to examine.
Now extracting data from listing 351 out of 415.
Good! There are 414 listings left to examine.
Now extracting data from listing 352 out of 414.
Good! There are 413 listings left to examine.
Now extracting data from listing 353 out of 413.
Good! There are 412 listings left to examine.
Now 

Good! There are 334 listings left to examine.
Now extracting data from listing 49 out of 334.
Good! There are 333 listings left to examine.
Now extracting data from listing 50 out of 333.
Good! There are 332 listings left to examine.
Now extracting data from listing 51 out of 332.
Good! There are 331 listings left to examine.
Now extracting data from listing 52 out of 331.
Good! There are 330 listings left to examine.
Now extracting data from listing 53 out of 330.
Good! There are 329 listings left to examine.
Now extracting data from listing 54 out of 329.
Good! There are 328 listings left to examine.
Now extracting data from listing 55 out of 328.
Good! There are 327 listings left to examine.
Now extracting data from listing 56 out of 327.
Good! There are 326 listings left to examine.
Now extracting data from listing 57 out of 326.
Good! There are 325 listings left to examine.
Now extracting data from listing 58 out of 325.
Good! There are 324 listings left to examine.
Now extracting

You Shall Not Pass!
Now extracting data from listing 136 out of 248.
You Shall Not Pass!
Now extracting data from listing 137 out of 248.
You Shall Not Pass!
Now extracting data from listing 138 out of 248.
You Shall Not Pass!
Now extracting data from listing 139 out of 248.
You Shall Not Pass!
Now extracting data from listing 140 out of 248.
You Shall Not Pass!
Now extracting data from listing 15 out of 248.
You Shall Not Pass!
Now extracting data from listing 142 out of 248.
Good! There are 247 listings left to examine.
Now extracting data from listing 143 out of 247.
Good! There are 246 listings left to examine.
Now extracting data from listing 144 out of 246.
Good! There are 245 listings left to examine.
Now extracting data from listing 145 out of 245.
Good! There are 244 listings left to examine.
Now extracting data from listing 146 out of 244.
Good! There are 243 listings left to examine.
Now extracting data from listing 147 out of 243.
This appartment is no longer available.
Now

Good! There are 165 listings left to examine.
Now extracting data from listing 30 out of 165.
Good! There are 164 listings left to examine.
Now extracting data from listing 31 out of 164.
Good! There are 163 listings left to examine.
Now extracting data from listing 32 out of 163.
Good! There are 162 listings left to examine.
Now extracting data from listing 33 out of 162.
Good! There are 161 listings left to examine.
Now extracting data from listing 34 out of 161.
Good! There are 160 listings left to examine.
Now extracting data from listing 35 out of 160.
Good! There are 159 listings left to examine.
Now extracting data from listing 36 out of 159.
Good! There are 158 listings left to examine.
Now extracting data from listing 37 out of 158.
Good! There are 157 listings left to examine.
Now extracting data from listing 38 out of 157.
Good! There are 156 listings left to examine.
Now extracting data from listing 39 out of 156.
Good! There are 155 listings left to examine.
Now extracting

Good! There are 77 listings left to examine.
Now extracting data from listing 21 out of 77.
Good! There are 76 listings left to examine.
Now extracting data from listing 22 out of 76.
Good! There are 75 listings left to examine.
Now extracting data from listing 23 out of 75.
Good! There are 74 listings left to examine.
Now extracting data from listing 24 out of 74.
Good! There are 73 listings left to examine.
Now extracting data from listing 25 out of 73.
Good! There are 72 listings left to examine.
Now extracting data from listing 26 out of 72.
Good! There are 71 listings left to examine.
Now extracting data from listing 27 out of 71.
Good! There are 70 listings left to examine.
Now extracting data from listing 28 out of 70.
Good! There are 69 listings left to examine.
Now extracting data from listing 29 out of 69.
Good! There are 68 listings left to examine.
Now extracting data from listing 30 out of 68.
Good! There are 67 listings left to examine.
Now extracting data from listing 31

In [10]:
df_appartment.shape

(2338, 13)

In [15]:
df_appartment.head()

,link,title,agency,housing_type,city,details,rent,charges,rent_info,criteria,energy_rating,gas_rating,description
0,https://www.seloger.com/annonces/locations/app...,Location appartement 2 pièces Toulouse - appar...,zaf immobilier,Appartement,Toulouse,"location appartement 42,03m² t...",590,CC,None,"Surface de 42,03 m² ; Au 3ème étage ; 2 Pièces...",227.0,10.0,Votre agence ZAF IMMOBILIER vous propose: Char...
1,https://www.seloger.com/annonces/locations/app...,Location appartement 3 pièces Toulouse - appar...,eurim,Appartement,Toulouse,location appartement 85m² toul...,1650,CC,"\n Loyer : 1,650 € / mo...",Ascenseur ; Orientation Ouest ; Vue ; Refait À...,NaN,NaN,Wilson - Vue exceptionnelle WILSON - Superbe v...
2,https://www.seloger.com/annonces/locations/app...,Location appartement 4 pièces Toulouse - appar...,eurim,Appartement,Toulouse,"location appartement 90,7m² to...",1351,CC,"\n Loyer : 1,351 € / mo...","Ascenseur ; Orientation Est, Sud ; Vue ; Surfa...",148.0,7.0,COMPANS - PLACE DE L'EUROPE Compans-Caffarelli...
3,https://www.seloger.com/annonces/locations/app...,Location appartement 2 pièces Toulouse - appar...,sporting immobilier,Appartement,Toulouse,location appartement 42m² toul...,545,CC,None,Ascenseur ; 1 Balcon ; Surface de 42 m² ; Bâti...,61.0,14.0,APT 109\nSPORTING IMMOBILIER VOUS OFFRE L INTÉ...
4,https://www.seloger.com/annonces/locations/app...,Location appartement 3 pièces Toulouse - appar...,cdc habitat,Appartement,Toulouse,location appartement 66m² toul...,685,CC,None,Ascenseur ; 1 Balcon ; Orientation Ouest ; Sur...,66.0,15.0,SANS HONORAIRES. Dans immeuble sécurisée de 20...


In [16]:
df_appartment.to_csv('data_seloger_scraping_part1.csv',index=False)

Next steps for improvement:
- Rotate the user agent
- Rotate of proxies (proxy pool)
- Only extract the new listings to consolidate our data: Trier par date => afin d’avoir les offres récentes (le but étant de ne pas scraper 2 fois la même annonce, donc si 200 nouvelles annonces ont été publiées depuis hier, le scrap d’aujourd’hui doit s’arrêter quand il aura scraper ces 200 annonces et qu’il retrouve par la suite une annonce qu’il a déjà scrapé la veille)

#### References & code:
- https://medium.com/france-school-of-ai/web-scraping-avec-python-apprenez-%C3%A0-utiliser-beautifulsoup-proxies-et-un-faux-user-agent-d7bfb66b6556
- https://towardsdatascience.com/looking-for-a-house-build-a-web-scraper-to-help-you-5ab25badc83e
- https://medium.com/@ben.sturm/scraping-house-listing-data-using-selenium-and-beautiful-soup-1cbb94ba9492